In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
 if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && isFirstQuery && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <div className={styles.subgraphMessage}>
                  <SubgraphResults results={message.subgraphResults} />
                </div>
              )}
            </React.Fragment>
          ))}
        </div>

In [ ]:
from typing import Dict, List
from langgraph.graph import StateGraph, END

# Define your functions
def vector_db_search(query: str) -> str:
    # Implement your vector database search here
    return f"Vector DB result for: {query}"

def query_expansion(query: str) -> List[str]:
    # Implement your query expansion logic here
    return [f"{query} expanded 1", f"{query} expanded 2"]

def api_call(query: str) -> str:
    # Implement your API call here
    return f"API result for: {query}"

# Define the routing logic
def route_query(state):
    query = state['query']
    if len(query.split()) > 3:  # Simple routing logic, adjust as needed
        return "query_expansion"
    else:
        return "vector_db_search"

# Define the query expansion and API call logic
def expand_and_call(state):
    expanded_queries = query_expansion(state['query'])
    results = []
    for eq in expanded_queries:
        results.append(api_call(eq))
    return {"expanded_results": results}

# Define the aggregation logic
def aggregate_results(state):
    if "expanded_results" in state:
        # Aggregate expanded results
        combined = " ".join(state["expanded_results"])
        return {"response": f"Aggregated results: {combined}"}
    else:
        # Return vector DB result
        return {"response": state["result"]}

# Create the graph
workflow = StateGraph(name="QueryProcessor")

# Add nodes
workflow.add_node("route", route_query)
workflow.add_node("vector_db_search", lambda state: {"result": vector_db_search(state['query'])})
workflow.add_node("expand_and_call", expand_and_call)
workflow.add_node("aggregate", aggregate_results)

# Add edges
workflow.add_edge("route", "vector_db_search")
workflow.add_edge("route", "expand_and_call")
workflow.add_edge("vector_db_search", "aggregate")
workflow.add_edge("expand_and_call", "aggregate")
workflow.add_edge("aggregate", END)

# Set the entrypoint
workflow.set_entry_point("route")

# Compile the graph
app = workflow.compile()

# Example usage
query = "What is the capital of France?"
result = app.invoke({"query": query})
print(result['response'])

In [ ]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up the training data (noisy sine wave)
train_x_full = torch.linspace(0, 1, 100).to(device)
train_y_full = torch.sin(train_x_full * (2 * math.pi)) + torch.randn(train_x_full.size()).to(device) * math.sqrt(0.04)

# Define the GP Model
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean().to(device)
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel().to(device)
        ).to(device)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_and_evaluate_loocv():
    n_points = len(train_x_full)
    results = []
    mse_loss_list = []
    hyperparameter_history = []
    
    # Perform LOOCV
    for left_out_idx in tqdm(range(n_points), desc="LOOCV Progress"):
        # Create training set without the left-out point
        mask = torch.ones(n_points, dtype=bool)
        mask[left_out_idx] = False
        
        train_x = train_x_full[mask].reshape(-1, 1)
        train_y = train_y_full[mask]
        test_x = train_x_full[left_out_idx].reshape(1, 1)
        test_y = train_y_full[left_out_idx]
        
        # Initialize model and likelihood
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
        model = ExactGPModel(train_x, train_y, likelihood).to(device)
        
        # Training mode
        model.train()
        likelihood.train()
        
        # Optimize hyperparameters
        optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        
        # Training loop
        for i in range(50):  # 50 training iterations
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
        
        # Store hyperparameters
        hyperparameters = {
            'lengthscale': model.covar_module.base_kernel.lengthscale.item(),
            'outputscale': model.covar_module.outputscale.item(),
            'noise': model.likelihood.noise.item()
        }
        hyperparameter_history.append(hyperparameters)
        
        # Evaluation mode
        model.eval()
        likelihood.eval()
        
        # Make prediction
        with torch.no_grad():
            observed_pred = likelihood(model(test_x))
            pred_mean = observed_pred.mean.cpu().numpy().item()
            pred_std = torch.sqrt(observed_pred.variance).cpu().numpy().item()
        
        # Calculate MSE for this point
        test_actual = test_y.cpu().numpy().item()
        mse_loss = (pred_mean - test_actual) ** 2
        mse_loss_list.append(mse_loss)
        
        results.append({
            'index': left_out_idx,
            'x': test_x.cpu().numpy().item(),
            'actual': test_actual,
            'predicted': pred_mean,
            'std': pred_std,
            'mse': mse_loss,
            'hyperparameters': hyperparameters
        })
        
        if left_out_idx % 10 == 0:
            print(f"\nPoint {left_out_idx}:")
            print(f"MSE: {mse_loss:.4f}")
            print(f"Hyperparameters: {hyperparameters}")
    
    return results, mse_loss_list, hyperparameter_history

# Run LOOCV
results, mse_loss_list, hyperparameter_history = train_and_evaluate_loocv()

# Sort results by x-value for plotting
sorted_results = sorted(results, key=lambda x: x['x'])
x_values = [r['x'] for r in sorted_results]
actual_values = [r['actual'] for r in sorted_results]
predicted_values = [r['predicted'] for r in sorted_results]
std_values = [r['std'] for r in sorted_results]

# Plot predictions vs actual values
plt.figure(figsize=(15, 5))

# Prediction plot
plt.subplot(1, 2, 1)
plt.plot(x_values, actual_values, 'k*', label='Actual', markersize=4)
plt.plot(x_values, predicted_values, 'b-', label='Predicted')
plt.fill_between(
    x_values,
    [p - 2*s for p, s in zip(predicted_values, std_values)],
    [p + 2*s for p, s in zip(predicted_values, std_values)],
    color='blue', alpha=0.2, label='95% Confidence'
)
plt.xlabel('x')
plt.ylabel('y')
plt.title('LOOCV Predictions vs Actual Values')
plt.legend()

# MSE plot
plt.subplot(1, 2, 2)
plt.plot(x_values, mse_loss_list, 'r-', label='MSE')
plt.xlabel('x')
plt.ylabel('MSE')
plt.title(f'MSE at Each Point (Mean: {np.mean(mse_loss_list):.4f})')
plt.legend()

plt.tight_layout()
plt.show()

# Plot hyperparameter evolution
plt.figure(figsize=(12, 4))
hp_x = range(len(hyperparameter_history))
plt.plot(hp_x, [h['lengthscale'] for h in hyperparameter_history], label='Lengthscale')
plt.plot(hp_x, [h['outputscale'] for h in hyperparameter_history], label='Outputscale')
plt.plot(hp_x, [h['noise'] for h in hyperparameter_history], label='Noise')
plt.xlabel('Point Index')
plt.ylabel('Value')
plt.title('Hyperparameter Evolution During LOOCV')
plt.legend()
plt.show()

In [ ]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up the training data (noisy sine wave)
train_x_full = torch.linspace(0, 1, 100).to(device)
train_y_full = torch.sin(train_x_full * (2 * math.pi)) + torch.randn(train_x_full.size()).to(device) * math.sqrt(0.04)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean().to(device)
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel().to(device)
        ).to(device)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_with_looph():
    """Train a model using LeaveOneOutPseudoLikelihood"""
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    model = ExactGPModel(train_x_full, train_y_full, likelihood).to(device)
    
    # Training mode
    model.train()
    likelihood.train()
    
    # Use LeaveOneOutPseudoLikelihood
    looph = gpytorch.mlls.LeaveOneOutPseudoLikelihood(likelihood, model)
    
    # Optimize
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    
    losses = []
    hyperparameters = []
    
    for i in range(50):
        optimizer.zero_grad()
        output = model(train_x_full)
        loss = -looph(output, train_y_full)
        losses.append(loss.item())
        
        # Store hyperparameters
        hyperparameters.append({
            'lengthscale': model.covar_module.base_kernel.lengthscale.item(),
            'outputscale': model.covar_module.outputscale.item(),
            'noise': model.likelihood.noise.item()
        })
        
        loss.backward()
        optimizer.step()
    
    # Get LOO predictions
    model.eval()
    likelihood.eval()
    with torch.no_grad():
        output = model(train_x_full)
        looph_dist = looph.likelihood_objective(output, train_y_full)
        loo_means = looph_dist.mean.cpu()
        loo_vars = looph_dist.variance.cpu()
        
    return {
        'model': model,
        'likelihood': likelihood,
        'loo_means': loo_means,
        'loo_vars': loo_vars,
        'losses': losses,
        'hyperparameters': hyperparameters,
        'final_hyperparameters': hyperparameters[-1]
    }

def train_and_evaluate_loocv():
    """Perform traditional LOOCV"""
    n_points = len(train_x_full)
    results = []
    
    for left_out_idx in tqdm(range(n_points), desc="LOOCV Progress"):
        # Create training set without the left-out point
        mask = torch.ones(n_points, dtype=bool)
        mask[left_out_idx] = False
        
        train_x = train_x_full[mask].reshape(-1, 1)
        train_y = train_y_full[mask]
        test_x = train_x_full[left_out_idx].reshape(1, 1)
        test_y = train_y_full[left_out_idx]
        
        # Initialize and train model
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
        model = ExactGPModel(train_x, train_y, likelihood).to(device)
        
        # Training mode
        model.train()
        likelihood.train()
        
        # Use standard MLL for LOOCV
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        
        # Optimize
        optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        
        for i in range(50):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
        
        # Evaluation mode
        model.eval()
        likelihood.eval()
        
        # Predict
        with torch.no_grad():
            observed_pred = likelihood(model(test_x))
            pred_mean = observed_pred.mean.cpu().numpy().item()
            pred_var = observed_pred.variance.cpu().numpy().item()
            
        results.append({
            'index': left_out_idx,
            'x': test_x.cpu().numpy().item(),
            'actual': test_y.cpu().numpy().item(),
            'predicted': pred_mean,
            'variance': pred_var
        })
    
    return results

# Run both methods
print("Training model with LOOPH...")
looph_results = train_with_looph()

print("\nPerforming LOOCV...")
loocv_results = train_and_evaluate_loocv()

# Prepare data for plotting
x_values = train_x_full.cpu().numpy()
actual_values = train_y_full.cpu().numpy()

looph_means = looph_results['loo_means'].numpy()
looph_stds = torch.sqrt(looph_results['loo_vars']).numpy()

loocv_sorted = sorted(loocv_results, key=lambda x: x['index'])
loocv_means = np.array([r['predicted'] for r in loocv_sorted])
loocv_stds = np.array([np.sqrt(r['variance']) for r in loocv_sorted])

# Create comparison plots
plt.figure(figsize=(15, 10))

# Predictions comparison
plt.subplot(2, 1, 1)
plt.plot(x_values, actual_values, 'k*', label='Actual', markersize=4)
plt.plot(x_values, looph_means, 'b-', label='LOOPH Predictions')
plt.fill_between(x_values, 
                 looph_means - 2*looph_stds,
                 looph_means + 2*looph_stds,
                 color='blue', alpha=0.2, label='LOOPH 95% CI')
plt.plot(x_values, loocv_means, 'r--', label='LOOCV Predictions')
plt.fill_between(x_values,
                 loocv_means - 2*loocv_stds,
                 loocv_means + 2*loocv_stds,
                 color='red', alpha=0.2, label='LOOCV 95% CI')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Predictions Comparison: LOOPH vs LOOCV')
plt.legend()

# Training loss for LOOPH
plt.subplot(2, 1, 2)
plt.plot(looph_results['losses'], 'b-')
plt.xlabel('Iteration')
plt.ylabel('LOOPH Loss')
plt.title('LOOPH Training Loss')

plt.tight_layout()
plt.show()

# Plot hyperparameter evolution for LOOPH
plt.figure(figsize=(12, 4))
iterations = range(len(looph_results['hyperparameters']))
plt.plot(iterations, [h['lengthscale'] for h in looph_results['hyperparameters']], 
         label='Lengthscale')
plt.plot(iterations, [h['outputscale'] for h in looph_results['hyperparameters']], 
         label='Outputscale')
plt.plot(iterations, [h['noise'] for h in looph_results['hyperparameters']], 
         label='Noise')
plt.xlabel('Iteration')
plt.ylabel('Value')
plt.title('Hyperparameter Evolution During LOOPH Training')
plt.legend()
plt.show()

# Print summary statistics
print("\nFinal Hyperparameters (LOOPH):")
for param, value in looph_results['final_hyperparameters'].items():
    print(f"{param}: {value:.4f}")

# Compute correlation between predictions
correlation = np.corrcoef(looph_means, loocv_means)[0,1]
print(f"\nCorrelation between LOOPH and LOOCV predictions: {correlation:.4f}")

# Plot prediction differences
plt.figure(figsize=(10, 5))
prediction_diff = looph_means - loocv_means
plt.plot(x_values, prediction_diff, 'g-', label='LOOPH - LOOCV')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.xlabel('x')
plt.ylabel('Prediction Difference')
plt.title('Difference between LOOPH and LOOCV Predictions')
plt.legend()
plt.show()